In [1]:
import rpy2
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [2]:
%%R
library(dplyr)

R[write to console]: 
Attaching package: ‘dplyr’


R[write to console]: The following objects are masked from ‘package:stats’:

    filter, lag


R[write to console]: The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union




# Introduction to piping

## Review -- Piping with `dplyr`

You might be familiar with piping in R using the `%>%` operator from `dplyr`

In [3]:
%%R
pi %>% 
sqrt %>% 
round(2) %>% 
as.character

[1] "1.77"


## Making `pipeable` functions using `composable`

To get functions to be pipeable in Python, we need to wrap them in `pipeable` from the `composable` module

In [4]:
!pip install composable # Install if missing or in colab

In [5]:
from composable import pipeable

## Making some pipeable functions

Before I can recreate the R example, I need to make some pipeable functions.

## Making a pipeable `sqrt`

To make an existing function pipeable, I need to wrap or *decorate* it with `pipeable`.

In [6]:
import math as m
sqrt = pipeable(m.sqrt)

In [7]:
2 >> sqrt

1.4142135623730951

## Some common functions are not actually functions

Some Python "functions" are not actually functions, but type constructors.  Examples include `str`, `float`, `int`, `list`, etc.  This also includes the most powerful type constructor of them all, `type`

In [8]:
type(str)

type

In [10]:
wont_always_work = pipeable(float)
3.5 >> sqrt >> wont_always_work

1.8708286933869707

## Use a `lambda` to create a pipeable type conversion function

To be safe, we need to wrap type constructors in a lambda, THEN `pipeable`

In [11]:
toStr = pipeable(lambda n: str(n))

In [12]:
3.5 >> toStr

'3.5'

## Piping and multiple arguments

Piped in data is inserted **on the right**

In [17]:
test = pipeable(lambda a, b: f"first:{a} second:{b}")

In [18]:
test(1,2)

'first:1 second:2'

In [19]:
1 >> test(2)

'first:2 second:1'

In [20]:
2 >> test(1)

'first:1 second:2'

## Rearranging argument order

The default `round` function uses `round(number, digits)`

In [21]:
round(m.pi, 2)

3.14

For piping, it is more convenient to switch the order.

In [22]:
rnd = pipeable(lambda d, n: round(n, d))

In [23]:
m.pi >> rnd(2) 

3.14

## Recreating the R example

In [ ]:
m.pi >> sqrt >> rnd(2) >> toStr

'1.77'

In [24]:
(m.pi >> 
sqrt >> 
rnd(2) >> 
toStr)

'1.77'

## Hint 1: Wrap multi-line piped expressions in parentheses

In [ ]:
(m.pi >> 
 sqrt >> 
 rnd(2) >> 
 toStr)

'1.77'

## Hint 2: Put the pipes at the start of a line

In [ ]:
(m.pi 
 >> sqrt 
 >> rnd(2) 
 >> toStr
)

'1.77'

## Pipeable functions return functions when partially complete

Note that `pipeable` functions are curried, meaning they return functions if not provided with enough arguments.

In [ ]:
threeArgs= pipeable(lambda a, b, c: f"first:{a} second:{b} third:{c}")

In [26]:
threeArgs("Bob")

NameError: ignored

In [25]:
threeArgs("Bob", "Alice")

NameError: ignored

## We can save and call a partial functions 

In [27]:
bob = threeArgs("Bob")

NameError: ignored

In [28]:
bob(2,3)

NameError: ignored

In [29]:
bobAndAlice = bob("Alice")

NameError: ignored

In [ ]:
bobAndAlice(3)

'first:Bob second:Alice third:3'

## Example

Suppose that I round to two decimal places A LOT.  In this case it might be nice to have a specialized function

In [ ]:
rndToTwo = rnd(2)

In [ ]:
m.pi >> rndToTwo

3.14

<font color="red"><h1>Exercise 3</h1></font>

Here is a problem that you solved in a previous activity:

    The function `random` from the `random` module can be used to generate numbers between 0 and 1 at random. We want to return numbers between $a$ and $b$ at random, which can be accomplished using the formula $V = (b - a)*random() + a$.

    Write a lambda function that takes `a` and `b` as arguments are returns a number between `a` and `b` at random.
   
Note that we can name the parts the process as follows:

> b takeAway a >> times a random number >> subtract a

Let's redo this problem, but this time with piping; where we will make a pipeable function to perform each task.

In [40]:
from composable import pipeable
import random as r
takeAway = pipeable(lambda x, y: y - x )
timesRandom = pipeable(lambda x: x* r.random())
add = pipeable(lambda x, y: x + y )

In [38]:
def uniform(a, b):
    return ( b
            >> takeAway(a)
            >> timesRandom
            >> add(a)
            )

In [41]:
## A test function you should be able to pass when complete
assert all(1 <= uniform(1,2) <= 2 for i in range(10))